In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import copy
import h5py

import pyvista as pv
#pv.set_jupyter_backend('trame')
#pv.set_jupyter_backend(None) 
#pv.set_jupyter_backend('server')
#pv.set_jupyter_backend('static')

from coordinate import global_coord,coord_sys
from rim import Elliptical_rim,Rect_rim
from surface import PolySurf
from Vectorpy import vector



In [ ]:
class Geo():
    '''
    Basic optical element.
    '''
    def __init__(self,surf,rim,coord_sys=global_coord):
        '''basic parameters'''
        self.surf=surf
        self.rim=rim
        self.coord=coord_sys
        self.View_3D=None
        '''coordinates of the object'''
        self.points=vector()
        self.r_points=vector()
        self.g_points=vector()
        self.normal_v=vector()
        self.widget=pv.Plotter()
        

    def view(self):
        x,y,w=self.rim.sampling(10,10)
        del(w)
        z=self.surf.surface(x,y)
        x,y,z=self.coord._toGlobal_coord(x,y,z)
        points = np.c_[x.reshape(-1), y.reshape(-1), z.reshape(-1)]
        del(x,y,z)
        if self.View_3D!=None:
            self.widget.remove_actor(self.View_3D)
        else:
            cloud = pv.PolyData(points)
            self.View_3D = cloud.delaunay_2d()
        del(points)
        self.widget.add_mesh(self.View_3D,show_edges=True)
        self.widget.show()
        
    def sampling(self,Nx,Ny):
        # sampling
        self.points.x,self.points.y,self.weight=self.rim.sampling(Nx,Ny)
        # calculate surface in z.
        self.points.z=self.surf.surface(self.points.x,self.points.y)
        # to global coordinates
        self.g_points.x,self.g_points.y,self.g_points.z=self.coord._toGlobal_coord(self.g_points.x,self.g_points.y,self.g_points.z)
        # surface normal vectors.
        self.normal_v.x,self.normal_v.y,self.normal_v.z,self.N=self.surf.normal_vector(self.points.x,self.points.y)
        xyz=np.append([self.normal_v.x,self.normal_v.y],[self.normal_v.z],axis=0)
        # convert normal vector in global coordinate system
        self.normal_v.x,self.normal_v.y,self.normal_v.z=np.matmul(self.coord.mat_l_g,xyz)
        del(xyz)

    def to_coord(self,tar_coord):
        points_new=vector()
        points_new.x,points_new.y,points_new.z=tar_coord.Global_to_local(self.g_points.x,self.g_points.y,self.g_points.z)
        return points_new
    
    def vector_rotation(self,tar_coord):
        Nvector=copy.copy(self.normal_v)
        xyz=np.matmul(tar_coord.mat_g_l,xyz=np.append([self.normal_v.x,self.normal_v.y],[self.normal_v.z],axis=0))
        Nvector.x=xyz[0,:]
        Nvector.y=xyz[1,:]
        Nvector.z=xyz[2,:]
        return Nvector


In [ ]:
class Reflector(Geo):
    '''
    Reflector, perfect conductor
    '''
    def __init__(self,surf,rim,coord_sys=global_coord,loss=None):
        Geo.__init__(self,surf,rim,coord_sys)

In [ ]:
class lens(Geo):
    '''
    simple lens
    '''
    def __init__(self,surf,rim,coord_sys=global_coord,):
        pass

In [ ]:
Theta_0=0.927295218001612; # offset angle of MR;
Ls      = 12000.0;           # distance between focal point and SR
Lm      = 6000.0;            # distance between MR and SR;
L_fimag=18000+Ls;
F=20000;

defocus=[0,0,0]
angle_m2=[-(np.pi/2+Theta_0)/2,0,0] #  1. m2 and global co-ordinates
D_m2=[0,-Lm*np.sin(Theta_0),0]

angle_m1=[-Theta_0/2,0,0]          #  2. m1 and global co-ordinates
D_m1=[0,0,Lm*np.cos(Theta_0)]

angle_s=[0,np.pi,0];               #  3. source and global co-ordinates
D_s=[0,0,0];

angle_fimag=[-Theta_0,0,0];        #  4. fimag and global co-ordinates
defocus_fimag=[0,0,0];
defocus_fimag[2]=1/(1/F-1/(Ls+defocus[2]))+L_fimag;
defocus_fimag[1]=(F+L_fimag-defocus_fimag[2])/F*defocus[1];
defocus_fimag[0]=(F+L_fimag-defocus_fimag[2])/F*defocus[0];
D_fimag=[0,0,0]
D_fimag[0]=defocus_fimag[0];
D_fimag[1]=defocus_fimag[1]*np.cos(Theta_0)-np.sin(Theta_0)*(L_fimag-defocus_fimag[2]+Lm);
D_fimag[2]=-defocus_fimag[1]*np.sin(Theta_0)-np.cos(Theta_0)*(L_fimag-defocus_fimag[2]);

# 5. feed and global co-ordinate
'''
C=1/(1/Lm-1/F)+defocus[2]+Ls;
C=21000;
angle_f=[np.pi/2-defocus[1]/C,0,-defocus[0]/C]; 
D_f=[defocus[0],Ls+defocus[2]-Lm*np.sin(Theta_0),-defocus[1]];
'''
angle_f=[np.pi/2,0,0];    
D_f=[defocus[0],Ls+defocus[2]-Lm*np.sin(Theta_0),-defocus[1]]

surf_M1=PolySurf('coeffi_m1.surfc',R=3000)
coor_M1=coord_sys(D_m1,angle_m1,axes='xyz',ref_coord=global_coord)
rim_M1=Rect_rim([0,0],6000,6000)

surf_M2=PolySurf('coeffi_m2.surfc',R=3000)
coor_M2=coord_sys(D_m2,angle_m2,axes='xyz',ref_coord=global_coord)
rim_M2=Rect_rim([0,0],6000,6000)

In [ ]:
Ref_m1=Reflector(surf_M1,rim_M1,coord_sys=coor_M1)
Ref_m2=Reflector(surf_M2,rim_M2,coord_sys=coor_M2)
Ref_m1.sampling(100,100)
Ref_m2.sampling(100,100)
Ref_m1.view()
Ref_m2.view()
p.show()

In [ ]:
Ref_m1.view(p)
Ref_m2.view(p)
p.show()

In [ ]:
O2=vector()
O2.x=np.array([1.1,4.5])

In [ ]:
def test(d):
    d.__init__()
    del(d)

In [ ]:
test(O2)
O2.x

In [ ]:
O2.to_Tensor(T.device('cuda'))

In [ ]:
T.cuda.is_available()

In [ ]:
O2.x

In [27]:
import torch as T
x=T.ones((10001,13001),dtype=T.cdouble)

In [ ]:
x.to(T.device('cuda:0'))

In [ ]:
T.cuda.memory_allocated(0)/(1024**3)

In [ ]:
T.cuda.get_device_properties(0).total_memory/(1024**3)

In [ ]:
import numpy as np
x=T.tensor(np.linspace(-1,1,101))

In [ ]:
x=x.to('cuda')
x

In [29]:
x=T.tensor(np.array([[1,1,1,1,1],[2,2,2,2,2],[3,3,3,3,3]]).T).reshape((5,3,1))
x

tensor([[[1],
         [2],
         [3]],

        [[1],
         [2],
         [3]],

        [[1],
         [2],
         [3]],

        [[1],
         [2],
         [3]],

        [[1],
         [2],
         [3]]], dtype=torch.int32)

In [30]:
y=T.tensor([[1,3,4,5],
            [6,7,8,9],
            [10,11,12,13]])

In [31]:
C=x-y
C

tensor([[[  0,  -2,  -3,  -4],
         [ -4,  -5,  -6,  -7],
         [ -7,  -8,  -9, -10]],

        [[  0,  -2,  -3,  -4],
         [ -4,  -5,  -6,  -7],
         [ -7,  -8,  -9, -10]],

        [[  0,  -2,  -3,  -4],
         [ -4,  -5,  -6,  -7],
         [ -7,  -8,  -9, -10]],

        [[  0,  -2,  -3,  -4],
         [ -4,  -5,  -6,  -7],
         [ -7,  -8,  -9, -10]],

        [[  0,  -2,  -3,  -4],
         [ -4,  -5,  -6,  -7],
         [ -7,  -8,  -9, -10]]])

In [32]:
C**2

tensor([[[  0,   4,   9,  16],
         [ 16,  25,  36,  49],
         [ 49,  64,  81, 100]],

        [[  0,   4,   9,  16],
         [ 16,  25,  36,  49],
         [ 49,  64,  81, 100]],

        [[  0,   4,   9,  16],
         [ 16,  25,  36,  49],
         [ 49,  64,  81, 100]],

        [[  0,   4,   9,  16],
         [ 16,  25,  36,  49],
         [ 49,  64,  81, 100]],

        [[  0,   4,   9,  16],
         [ 16,  25,  36,  49],
         [ 49,  64,  81, 100]]])

In [33]:
T.sum(C**2,axis=-2)

tensor([[ 65,  93, 126, 165],
        [ 65,  93, 126, 165],
        [ 65,  93, 126, 165],
        [ 65,  93, 126, 165],
        [ 65,  93, 126, 165]])

In [5]:
x

array([0., 1., 2., 3., 4., 5., 6., 7.])

In [28]:
T.pi

3.141592653589793